<img alt="Colaboratory logo" width="15%" src="https://github.com/pedrohmpaiva/portfolio/blob/main/banner.png?raw=true">

*por: [Pedro Henrique M. Paiva](https://www.linkedin.com/in/pedro-henrique-paiva/)*  

---

# Detecção de Fraude em Cartões de Crédito

Neste projeto,suregido pelo curso [DSNP - Sigmoidal](https://sigmoidal.ai/), iremos abordar o problema das fraudes em cartões de crédito, uma das principais preocupações das instituições financeiras como bancos e *fintechs*. Segundo [esta matéria](https://valorinveste.globo.com/produtos/servicos-financeiros/noticia/2020/02/12/brasil-e-2o-pais-da-america-latina-com-mais-fraudes-no-cartao-em-compras-online.ghtml), em 2020 o Brasil foi o 2º país da américa latina com mais fraudes em compras online e no primeiro semestre de 2021 [teve alta de quase 33% nas tentativas de fraude com cartão de crédito](https://www.infomoney.com.br/minhas-financas/brasil-teve-alta-de-quase-33-nas-tentativas-de-fraude-com-cartao-de-credito-no-1-semestre-mostra-estudo/).

<p align=center>
<img src="https://img.freepik.com/fotos-gratis/homem-hacker-no-laptop_144627-25527.jpg?w=1380&t=st=1650512651~exp=1650513251~hmac=71e86ae27a9ebabffa336182bf8bce35a568a1f0261b81339b5bc789ed16efce" width="90%"></p>

Dentre essas fraudes, aquelas envolvendo cartões de crédito são de grande relevância uma vez que a sua não-detecção acaretará em prejuízos consideráveis, tanto para o consumidor quanto para a instituição financeira.

Por todos esses motivos, o investimento na área de detecção de fraudes por meio de Inteligência Artificial vem crescendo a cada ano, representando uma grande oportunidade em *Data Science*. 

Dispondo de grandes volumes de dados como base histórica, um algoritmo de machine learning apenas um pouco melhor que os anteriores já representa uma economia de milhões de Reais. E esse é o desafio, aprimorar cada vez mais o uso de algoritmos visando inibir ou evitar transações fraudulentas.

## Importando os Dados

Os dados que usaremos neste projeto foram disponibilizados por algumas empresas européias de cartão de crédito. O *dataset* representa as operações financeiras que aconteceram no período de dois dias, onde foram classificadas 492 fraudes em meio a quase 290 mil transações.
<p align=center>
    <img src="https://img.freepik.com/fotos-gratis/cartao-de-plastico-do-banco-pendurado-no-gancho-de-pesca-closeup-conceito-de-fraude-na-internet_151013-35671.jpg?w=1800" width="50%" align = right></p>


Como você pode notar, este é um conjunto de dados extremamente desbalanceado, onde as fraudes representam apenas 0,17% do total.

Outro detalhe é que as *features* são todas numéricas, e foram descaracterizadas (por preservação da privacidade e da segurança). Assim, os nomes das colunas são representados por $[V1, V2, V3 \dots, V28]$ 



[Na página original dos dados](https://www.kaggle.com/mlg-ulb/creditcardfraud), também é informado que as variáveis passaram por uma transformação conhecida como Análise de Componentes Principais (*Principal Component Analysis* - PCA).

A PCA permite a redução da dimensionalidade enquanto mantém o maior número possível de informações. Para conseguir isso, o algoritmo encontra um conjunto novo de recursos - os chamados **componentes**.

Esses componentes são em número menor or igual às variáveis originais. No caso deste projeto, os componentes achados pela transformação da PCA são as próprias colunas $[V1, V2, V3 \dots, V28]$.

---

In [563]:
import warnings
warnings.filterwarnings("ignore")

In [564]:
# importando os pacotes necessários
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler
from sklearn.linear_model    import LogisticRegression
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import classification_report
from sklearn.metrics         import r2_score
from sklearn.metrics         import confusion_matrix
from sklearn.metrics         import mean_squared_error
from sklearn.tree            import DecisionTreeClassifier, plot_tree

from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# importando os dados para um DataFrame

file_path = "https://www.dropbox.com/s/b44o3t3ehmnx2b7/creditcard.csv?dl=1"

df = pd.read_csv(file_path)
df.head(10)

---
Com os dados importados para dentro de uma estrutura *Dataframe* - e não havendo a necessidade de mais nenhum ajuste ou configuração nesta etapa, pode-se iniciar uma análise exploratória dos dados.

# Análise Exploratória

O objetivo desta etapa é ver o que inicialmente os dados tem a nos oferecer em insights, verificar a distribuição das classes, verificar se há valores ausentes, etc.

In [ ]:
#Verificando as 5 primeiras entradas e a dimensão do dataset

print('*** Dimensionalidade do dataset {} ***'.format(df.shape))
df.head()

In [ ]:
#Verificando algumas informações sobre o dataset e suas features

df.info()

In [ ]:
#Verificando se há valores ausentes

df.isnull().sum()

In [ ]:
#Resumo estatístico do dataset

df.describe()

In [ ]:
#Verificando a distribuição de classes

semFraude = df['Class'].value_counts()[0]
fraude = df['Class'].value_counts()[1]
print('Transações normais: {}'.format(semFraude))
print('Transações fraudulentas: {}'.format(fraude))

In [ ]:
#Cores

preto_titulo = '#363434'
preto_elementos = '#5c5757'
azul = '#1f4287' # original da paleta: '#62929a' alternativa: #4b89ac
cinza = '#efecec'

In [ ]:
#Plotando um grafico da distribuição das classes

fig, ax = plt.subplots(figsize=(10,6))
sns.countplot(df['Class'], color= azul)

ax.set_title('Gráfico de distribuição das classes', 
             fontsize = 20, 
             color = preto_titulo, 
             loc = 'left', 
             pad= 60, 
             fontweight= 'bold')

ax.text(-0.5,320000, 
        'Ocorrências de transações fraudulentas\ne não-fraudulentas', 
        fontsize = 13, color = preto_elementos)

ax.set_xticklabels(labels=['Normal', 'Fraude'])
ax.set_xlabel('Transações',  fontsize = 12, color= preto_elementos)
ax.set_ylabel('Ocorrências', fontsize = 12, color= preto_elementos)

ax.spines['top'].set_visible(False)

ax.annotate(semFraude, 
            xy=(-0.11, 290000), 
            color = preto_elementos, 
            fontsize= 20,
            fontweight='light')
ax.annotate(fraude, 
            xy=(0.93, 9000), 
            color = preto_elementos, 
            fontsize= 20,
            fontweight='light')

ax.spines['right'].set_visible(False)

## Plotando os histogramas da variável "Time" (Fraude e não-fraude)

A variável "Time" representa o tempo (em segundos) decorrido entre a primeira transação do dataset e a atual.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols = 1, figsize=(12,7))


ax[0].hist(df.Time[df.Class == 1], bins = 30, color= azul, rwidth= 0.9)
ax[0].set_title('Transações Fraudulentas',fontsize = 20, 
                color = preto_titulo, 
                loc = 'left', 
                pad= 60, 
                fontweight= 'bold')
ax[0].set_xlabel('Tempo Decorrido (s)', fontsize = 12, color= preto_elementos)
ax[0].set_ylabel('Nº de Ocorrências'  , fontsize = 12, color= preto_elementos)
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)


ax[1].hist(df.Time[df.Class == 0], bins = 30, color= azul, rwidth= 0.9)
ax[1].set_title('Transações Normais',
                fontsize = 20,
                color = preto_titulo, 
                loc = 'left', 
                pad= 60, 
                fontweight= 'bold')
ax[1].set_xlabel('Tempo Decorrido (s)', fontsize = 12, color= preto_elementos)
ax[1].set_ylabel('Nº de Ocorrências'  , fontsize = 12, color= preto_elementos)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)

fig.tight_layout(pad = 2)

## Plotando os histogramas da variável "Amount" (Fraude e não-fraude)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols = 1, figsize=(12,10))


ax[0].hist(df.Amount[df.Class == 1], bins = 30, color= azul, rwidth= 0.9)

ax[0].text(1900,300, "Média ${:.2f}".format(df.Amount[df.Class == 1].mean()),
           fontsize = 12, color= preto_elementos)

ax[0].set_title('Transações Fraudulentas',
                fontsize = 20,
                color = preto_titulo, 
                loc = 'left', 
                pad= 60, 
                fontweight= 'bold')
ax[0].set_xlabel('Quantidade ($)'   , fontsize = 12, color= preto_elementos)
ax[0].set_ylabel('Nº de Ocorrências', fontsize = 12, color= preto_elementos)

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)



ax[1].hist(df.Amount[df.Class == 0], bins = 30, color= azul, rwidth= 0.9)

ax[1].text(23000,250000, "Média ${:.2f}".format(df.Amount[df.Class == 0].mean()),
           fontsize = 12, color= preto_elementos)

ax[1].set_title('Transações Normais',
                fontsize = 20,
                color = preto_titulo, 
                loc = 'left', 
                pad= 60, 
                fontweight= 'bold')

ax[1].set_xlabel('Quantidade ($)'   , fontsize = 12, color= preto_elementos)
ax[1].set_ylabel('Nº de Ocorrências', fontsize = 12, color= preto_elementos)

ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)



fig.tight_layout(pad = 2)

As transações fraudulentas tem menos ocorrências neste conjunto de dados, como vimos anteriormente. Talvez por isto a média das transações fraudulentas se mostre maior do que as transações normais, como foi visto no gráfico acima. 


Vejo que a maioria das transações normais feitas durante esta coleta de dados estão focadas entre 0 e 5000 dólares, com uma média de 88 dólares.

In [ ]:
# Boxplot das transações "fraude" e "normais"

fig, ax = plt.subplots(figsize=(6,10))

sns.boxplot(df.Class, df.Amount, ax=ax, color= azul)
plt.ylim((-20, 1500))
plt.xticks([0, 1], ['Normal', 'Fraude'],fontsize = 12, color= preto_elementos)

ax.set_title('Boxplot - Transações',
                fontsize = 20,
                color = preto_titulo, 
                loc = 'left', 
                pad= 60, 
                fontweight= 'bold')
ax.set_ylabel('Quantidade ($)',fontsize = 12, color= preto_elementos)
ax.set_xlabel('')

plt.tight_layout(pad = 2)

***Fiz um corte limitando a quantidade até 1500 dólares com o objetivo de uma visualização mais agradável***

---

In [ ]:
#Matriz de correlação

corr_  = df.corr(method= 'spearman')

pd.DataFrame(corr_).style.background_gradient(cmap='coolwarm')

### Conclusões da análise exploratória


* O dataset não possui valores ausentes para serem tratados.
* As classes que representam transações fraudulentas e não-fraudulentas estão totalmente desbalanceadas e isso precisará ser tratado. Se usarmos o conjunto de dados assim mesmo, o modelo não irá generalizar bem o padrão da classe que representa fraude, ficando muito bom apenas em detectar transações não fraudulentas.
* As variáveis "Time" e "Amount" estão em grandezas diferentes do restante das outras *features*. Estas duas colunas precisarão ser normalizadas, já que todo o *dataset* se encontra também normalizado. Caso isso não ocorra, acarretará em dificuldades na hora de treinar o modelo.


# Preparação dos Dados
---

Aqui, os dados serão preparados para posteriormente haver a criação do modelo e treiná-lo. Nesta etapa irei:

* Normalizar os dados que ainda não haviam sido pré-processados (`Time` e `Amount`)
* Dividir o conjunto de dados entre treino e teste
* Balancear o conjunto de dados

In [ ]:
#Criando uma cópia do dataframe

df_limpo = df.copy()

In [ ]:
#Importando o algoritmo que usarei para normalizar os dados

scaler = StandardScaler()

In [ ]:
#Atribuindo ao modelo as variáveis "Time" e "Amount" e criando novas variáveis dentro do dataframe

df_limpo['ss_time'] = scaler.fit_transform(df_limpo[['Time']])
df_limpo['ss_amount'] = scaler.fit_transform(df_limpo[['Amount']])

In [ ]:
#Deletando as variáveis não-normalizadas

df_limpo.drop(['Time', 'Amount'], axis= 1, inplace= True)

In [ ]:
#Conferindo o resultado

df_limpo.head(10)

In [ ]:
#Separando as variáveis depedentes e independente

X = df_limpo.drop('Class', axis = 1)
y = df_limpo['Class']

## Undersampling


Como o conjunto de dados está totalmente desbalanceado onde as transações que não são fraude representam a maior parte das ocorrências, é preciso aplicar técnicas de balanceamento antes de treinar nosso modelo

A técnica que eu escolhi é chamada de "Undersampling". O método NearMiss da biblioteca [imblearn](https://imbalanced-learn.org/stable/install.html#) é um algoritmo de "Undersampling" onde ele basicamente pega a quantidade de ocorrencias que não são fraude e reduz a mesma quantidade das transações que são fraude. Isto é, o algoritmo faz um corte nos dados de modo que a classe com maior ocorrência se reduz e fica igual a classe que tem menos ocorrências. Se treinarmos o nosso modelo simplesmente com dados desbaçanceados, ele vai ficar muito melhor em acertar transações que não são fraude do que as que são. Por isso, balancear os dados nesse cenário é altamente necessário!

Além do Undersampling, testei com Oversampling e também aplicando ambas as técnicas. Porém, obtive os melhores resultados usando o Undersampling.



In [ ]:
#Importando o modelo de Undersampling

nm = NearMiss()

In [ ]:
#Ajustando o modelo com os dados e os atribuindo a novas variáveis

X_under, y_under = nm.fit_resample(X, y)

In [ ]:
#Plotando um antes e depois dos dados

fig, ax = plt.subplots(ncols= 2, figsize = (15,5))


sns.countplot(df.Class, ax = ax[0], color= azul)
ax[0].set_title('Antes do Undersampling',
                fontsize = 15,
                color = preto_titulo, 
                pad= 10, 
                fontweight= 'bold')
ax[0].set_xlabel('Transação',fontsize = 12, color= preto_elementos)
ax[0].set_ylabel('Quantidade de ocorrências',fontsize = 12, color= preto_elementos)
ax[0].set_xticklabels(labels = ['Normal','Fraude'],fontsize = 12, color= preto_elementos)


sns.countplot(y_under, ax = ax[1], color= azul)
ax[1].set_title('Depois do Undersampling',
                fontsize = 15,
                color = preto_titulo, 
                pad= 10, 
                fontweight= 'bold')
ax[1].set_xlabel('Transação',fontsize = 12, color= preto_elementos)
ax[1].set_ylabel('Quantidade de ocorrências',fontsize = 12, color= preto_elementos)
ax[1].set_xticklabels(labels = ['Normal','Fraude'],fontsize = 12, color= preto_elementos)




# Modelo de Machine Learning
---
Com os dados prontos, é hora de construir nossos modelos. Optei por construir dois modelos afim de compará-los posteriormente. Escolhi usar uma regressão logística e uma árvore de decisão e ver como ambos se saem neste cenário.

In [ ]:
#Separando os dados entre treino e teste

X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)

In [ ]:
#Criando um modelo de Regressão Logística e ajustando-o

modelo = LogisticRegression()
modelo.fit(X_train, y_train)

In [ ]:
#Previsão dos resultados do modelo de Regressão Logística

y_pred = modelo.predict(X_test)

In [ ]:
#Criando um novo modelo de árvore de decisão

arvore = DecisionTreeClassifier(min_samples_leaf= 2, random_state= 10)
arvore.fit(X_train, y_train)

In [ ]:
#Previsão dos resultados usando árvore de decisão e avaliando-a usando o erro médio quadrado

y_pred_arvore = arvore.predict(X_test)

print("Erro médio quadrado: {}\n".format(np.sqrt(mean_squared_error(y_test, y_pred_arvore))))


Nos hiperparâmetros da árvore de decisão, afim de evitar um possível overfit existem dois parâmetros que ajudam: `max_depth` e o `min_samples_leaf`. O primeiro, limita a profundidade da nossa árvore, ou seja, quantas camadas ela terá. Já o segundo, ao invés de focar na profundidade, nele é limitado quantas folhas terão em cada camada.

Se não colocarmos hiperparâmetro nenhum, a árvore irá treinar o modelo até a profundidade que achar melhor e geralmente até ter uma folha em cada camada. Isso acaba se tornando muito sensível em casos de outliers e também a probabilidade de haver um overfit se torna maior.

O padrão que escolhi usar foi o `min_samples_leaf`, acabei tendo um resultado um pouco melhor do que o `max_depth`.Usei como métrica o erro médio quadrado, quanto menor o número, melhor. Aqui vão alguns resultados de testes que fiz:



`max_depth`
* => none: 0.24680702093691814
* => 20: 0.2759386380695814
* => 17: 0.2665820508731144
* => 18: 0.23629973222959152 - Profundidade Ideal
* => 16: 0.30227563311592703


`min_samples_leaf`
* => 1: 0.23629973222959152
* => 2: 0.18850197591499637 - Quantidade de folhas ideal
* => 3: 0.23629973222959152


Logo abaixo, um plot de como ficou a nossa árvore de decisão

In [ ]:
fig, ax = plt.subplots(figsize = (20,20))
ax = plot_tree(arvore, feature_names= X_train.columns)

## Avaliando o desempenho do modelo


In [ ]:
#Métricas do modelo - regressão logística
print(classification_report(y_test, y_pred))

In [ ]:
#Métricas - árvore de decisão
print(classification_report(y_test, y_pred_arvore))

In [ ]:
#Plotando matriz de confusão - RL

print('\n', classification_report(y_test, y_pred))

matriz = confusion_matrix(y_test, y_pred)
sns.heatmap(matriz, square=True, annot=True, cbar=False, cmap= 'Blues')

plt.title('Matriz de confusão - Regressão Logística',
          fontsize = 12,
          color = preto_titulo, 
          pad= 20, 
          fontweight= 'bold')

plt.xlabel('Previsão do modelo',fontsize = 12, color= preto_elementos)
plt.ylabel('Valor verdadeiro'  ,fontsize = 12, color= preto_elementos)

plt.show()

In [ ]:
#Plotando matriz de confusão - DT

print('\n', classification_report(y_test, y_pred_arvore))
print("Erro médio quadrado: {}\n".format(np.sqrt(mean_squared_error(y_test, y_pred_arvore))))
matriz = confusion_matrix(y_test, y_pred_arvore)
sns.heatmap(matriz, square=True, annot=True, cbar=False, cmap= 'Blues')

plt.title('Matriz de confusão - Decision Tree',
          fontsize = 12,
          color = preto_titulo, 
          pad= 20, 
          fontweight= 'bold')
plt.xlabel('Previsão do modelo',fontsize = 12, color= preto_elementos)
plt.ylabel('Valor verdadeiro'  ,fontsize = 12, color= preto_elementos)

plt.show()

## Conclusão
---

Os dois modelos escolhidos tiveram resultados bem parecidos, porém a *DesicionTree* se saiu um pouco melhor ao observarmos as métricas do `classification_report`. 


A diferença entre a *LogisticRegression* e a *DecisionTree* em termos de resultados é que a árvore errou uma previsão da classe "fraude" a menos que a regressão logística. 


Um outro fator a ser considerado é a quantidade de falsos positivos, ou seja, aquelas vezes em que você tentou fazer uma compra e teve seu cartão bloqueado preventivamente. Neste cenário, seria mais interessante focar em acertar mais as transações fraudulentas do que as não-fraudulentas, em outras palavras, melhor um falso positivo em uma transação normal ser fraude do que uma transação fraudulenta ser considerada como normal.
